# Corretoras e Imobiliarias

Vamos pegar as tabelas de todos os corretores e imobiliarias do rio de janeiro

## Recuperando tabela de consultores/imobiliaris

### Importando Dependências

In [34]:
import time
import pandas as pd
import pyautogui
pyautogui.PAUSE = 1
import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from io import StringIO

### Lista de Bairros

In [56]:
lista_bairros_zona_norte = [
    "Acari", "Anchieta", "Barros Filho", "Bento Ribeiro", "Brás de Pina", "Campinho", "Cavalcanti", "Cascadura", "Coelho Neto", "Colégio",
    "Complexo do Alemão", "Cordovil", "Costa Barros", "Engenheiro Leal", "Engenho da Rainha", "Guadalupe", "Honório Gurgel", "Inhaúma", "Irajá",
    "Jardim América", "Madureira", "Marechal Hermes", "Oswaldo Cruz", "Parada de Lucas", "Parque Anchieta", "Parque Colúmbia", "Pavuna", "Penha",
    "Penha Circular", "Quintinho Bocaiuva", "Ricardo de Albuquerque", "Rocha Miranda", "Tómas Coelho", "Turiaçu", "Vaz Lobo", "Vicente de Carvalho",
    "Vigário Geral", "Vila da Penha", "Vila Kosmos", "Vista Alegre" 
]
lista_bairros_ilha_gov_leopoldina = [
    "Bonsucesso", "Bancários", "Cacuia", "Cidade Universitária", "Cocotá", "Freguesia", "Galeão", "Jardim Carioca", "Jardim Guanabara", "Maré",
    "Moneró", "Olaria", "Pitangueiras", "Portuguesa", "Praia da Bandeira", "Ramos", "Ribeira", "Tauá", "Zumbi" 
]
lista_bairros_tijuca = [
    "Alto da Boa Vista", "Andaraí", "Grajaú", "Maracanã", "Praça da Bandeira", "Tijuca", "Vila Isabel", "Barra da Tijuca"
]
lista_bairros_meier = [
    "Abolição", "Agua Santa", "Cachambi", "Del Castilho", "Encantado", "Engenho de Dentro", "Engenho Novo", "Higienópolis", "Jacaré", "Jacarezinho",
    "Lins de Vasconcelos", "Manguinhos", "Maria da Graça", "Méier", "Piedade", "Pilares", "Riachuelo", "Rocha", "Sampaio", "São Francisco Xavier",
    "Todos os Santos",  
]

lista_bairros = lista_bairros_zona_norte + lista_bairros_ilha_gov_leopoldina + lista_bairros_tijuca + lista_bairros_meier
dict_bairros = {}

### Baixar tabela por bairro

In [36]:
def baixar_tabela_por_bairro(bairro):
    url = 'https://www.crecirj.conselho.net.br/form_pesquisa_cadastro_geral_site.php'
    driver = webdriver.Edge()

    # Maximize a janela do navegador
    driver.maximize_window()
    
    #Abrir pagina
    driver.get(url)
    
    # Localiza os campos do formulário e preenche-os
    campo_bairro = driver.find_element("id", "input-27")
    campo_bairro.send_keys(bairro)

    # Envie o formulário
    campo_bairro.send_keys(Keys.RETURN)

    # Aguarde um tempo para a página carregar completamente
    time.sleep(4)

    # Pular para o final da pagina
    for _ in range(25):  
        pyautogui.press('down')  # Simula pressionar a tecla de seta para baixo
        time.sleep(0.2) 

    # Mova o cursor do mouse para o ponto médio do elemento usando PyAutoGUI
    pyautogui.moveTo(171, 979)

    # Simule um clique do mouse usando PyAutoGUI
    pyautogui.click()

    time.sleep(1)

    pyautogui.click()

    time.sleep(6)

    try:
        # Encontra a tabela na página
        table = driver.find_elements('css selector', 'table')
        table_html = table[0].get_attribute('outerHTML')
    except IndexError:
        print(f"Não existe nenhuma corretora/imobiliaria em {bairro}!")
        print(f"Verifique se o nome do bairro esta certo! {bairro}\n")
        driver.quit()
        return pd.DataFrame()
    else:
        df = pd.read_html(StringIO(table_html))[0]  # Este método retorna uma lista de DataFrames, portanto, pegamos o primeiro
        driver.quit()
        print(bairro, len(df),end='; ')
        return df

### Baixando as tabelas de todos os bairros

In [57]:
def baixar_tabela_todos_bairros():
    for bairro in lista_bairros:
        tabela_bairro = baixar_tabela_por_bairro(bairro)
        dict_bairros.update({bairro:tabela_bairro})

baixar_tabela_todos_bairros()

Jardim Guanabara 397; Maré 282; Moneró 40; Olaria 361; Pitangueiras 30; Portuguesa 147; Praia da Bandeira 19; Ramos 222; Ribeira 81; Tauá 114; Zumbi 13; Alto da Boa Vista 78; Andaraí 288; Grajaú 523; Maracanã 327; Praça da Bandeira 122; Tijuca 7666; Vila Isabel 844; Barra da Tijuca 4762; Abolição 89; Agua Santa 28; Cachambi 439; Del Castilho 171; Encantado 122; Engenho de Dentro 397; Engenho Novo 331; Higienópolis 133; Jacaré 3602; Jacarezinho 5; Lins de Vasconcelos 197; Manguinhos 121; Maria da Graça 63; Méier 1020; Piedade 262; Pilares 207; Riachuelo 135; Rocha 435; Sampaio 73; São Francisco Xavier 28; Todos os Santos 241; 

### Baixando apenas um item

In [ ]:
#tabela_bairro_tijuca = baixar_tabela_por_bairro("Tijuca")

#dict_bairros.update({"Tijuca":tabela_bairro_tijuca})

### Criando uma cópia para preservar o dicionário original

In [58]:
dict_bairros_edit = dict_bairros.copy()
print(len(dict_bairros_edit))

40


### Removendo a natureza juridica do campo nome e adicionando campo bairro

In [59]:
def editar_campos_empresa():
    for bairro, tb in dict_bairros_edit.items():
        try:
            tb.drop(columns=['Unnamed: 5'], inplace=True)
        except KeyError:
            pass
        
        for index, linha in tb.iterrows():
            nome = linha["Identificação"]
            tb.at[index, "Bairro"] = bairro
            
            if " PJ " in nome:
                novo_nome = nome.replace(" PJ P", "").replace(" PJ ", "")
                tb.at[index, "Identificação"] = novo_nome
                tb.at[index, "Natureza"] = "PJ P"
            elif " PF " in nome:
                novo_nome = nome.replace(" PF P", "").replace(" PF ", "")
                tb.at[index, "Identificação"] = novo_nome
                tb.at[index,"Natureza"] = "PF P"
        print("Bairro:", bairro)

editar_campos_empresa()

Bairro: Jardim Guanabara
Bairro: Maré
Bairro: Moneró
Bairro: Olaria
Bairro: Pitangueiras
Bairro: Portuguesa
Bairro: Praia da Bandeira
Bairro: Ramos
Bairro: Ribeira
Bairro: Tauá
Bairro: Zumbi
Bairro: Alto da Boa Vista
Bairro: Andaraí
Bairro: Grajaú
Bairro: Maracanã
Bairro: Praça da Bandeira
Bairro: Tijuca
Bairro: Vila Isabel
Bairro: Barra da Tijuca
Bairro: Abolição
Bairro: Agua Santa
Bairro: Cachambi
Bairro: Del Castilho
Bairro: Encantado
Bairro: Engenho de Dentro
Bairro: Engenho Novo
Bairro: Higienópolis
Bairro: Jacaré
Bairro: Jacarezinho
Bairro: Lins de Vasconcelos
Bairro: Manguinhos
Bairro: Maria da Graça
Bairro: Méier
Bairro: Piedade
Bairro: Pilares
Bairro: Riachuelo
Bairro: Rocha
Bairro: Sampaio
Bairro: São Francisco Xavier
Bairro: Todos os Santos


### Exibindo quantos corretores/imobiliarias tem em cada bairro

In [60]:
def preencher_nan_com_vazio(df):
    # Iterar sobre todas as células do DataFrame
    for coluna in df.columns:
        for indice, valor in df[coluna].items():
            if pd.isna(valor):  # Verificar se o valor é NaN
                df.at[indice, coluna] = ""  # Substituir NaN por uma string vazia
    return df
new_dict = {}
for bairro, tb in dict_bairros_edit.items():
    novo_df = preencher_nan_com_vazio(tb)
    new_dict.update({bairro: novo_df})

dict_bairros_edit = new_dict.copy()

### Exibindo quantas corretoras/imobiliarias estão ativas e inativas

In [ ]:
ativos_qnt = 0
inativos_qnt = 0

def verifica_aitvos_inativos(dataframe):
    ativos = 0
    inativos = 0
    for index, linha in dataframe.iterrows():
        if "PJ" in  linha["Natureza"]:
            if linha["Situação"] == "ATIVO":
                ativos += 1 
            else: 
                inativos += 1
    return (ativos, inativos)

def contar_ativos_inativos_todos_bairros():
    for bairro, tb in dict_bairros_edit.items():
        novo_ativos_qnt, novo_inativos_qnt = verifica_aitvos_inativos(tb)
        ativos_qnt += novo_ativos_qnt
        inativos_qnt += novo_inativos_qnt

    print("Ativos", ativos_qnt)
    print("Inativos", inativos_qnt)
    print("Total", ativos_qnt + inativos_qnt)

#contar_ativos_inativos_todos_bairros()

### Filtrando Corretoras/Imobiliarias  (Situação = "ATIVO" e Certidão de Regularidade = "REGULAR")

In [ ]:
def bairros_ativos_disponiveis(bairros):
    novo_dict = {}
    for bairro_nome, bairro_tabela in bairros.items():
        print(bairro_nome)
        if not bairro_tabela.empty:
            nova_tabela = bairro_tabela.loc[bairro_tabela["Situação"] == "ATIVO"]
            nova_tabela = nova_tabela.loc[bairro_tabela["Certidão de Regularidade"] == "REGULAR"]
            novo_dict.update({bairro_nome:nova_tabela})
        else:
            print("Vazio",bairro_nome)
    return novo_dict.copy()

#print(len(dict_bairros_edit))
#dict_bairros_edit = bairros_ativos_disponiveis(dict_bairros_edit)
#print(len(dict_imobiliaria_ativa_disponivel["Tijuca"]))

## Descobrir CNPJ

### Verificar se o Edge esta maximizado

In [ ]:
import pygetwindow

def verificar_edge_maximizado():
    # Obter todas as janelas abertas atualmente
    janelas = pygetwindow.getAllWindows()

    # Procurar a janela do Microsoft Edge entre as janelas abertas
    for janela in janelas:
        if "Microsoft​ Edge" in janela.title:
            # Verificar se a janela está maximizada
            return janela.isMaximized
    else:
        print("A janela do Microsoft Edge não está aberta.")
        return False

# Exemplo de uso
# verificar_edge_maximizado()


### Iniciando Edge

In [ ]:
def iniciando_edge_pyautogui(url):
    pyautogui.press("win")
    pyautogui.write("edge")
    pyautogui.press("enter")
    
    time.sleep(1)
    # Ativar o modo de tela cheia pressionando windows + up
    esta_maximizado = verificar_edge_maximizado()
    if not esta_maximizado:
        pyautogui.hotkey('win', 'up')

    pyautogui.write(url)  
    pyautogui.press("enter")
    time.sleep(2)

def iniciando_edge_selenium(url):
    driver = webdriver.Edge()

    # Maximize a janela do navegador
    driver.maximize_window()
    
    #Abrir pagina
    driver.get(url)
    time.sleep(2)
    return driver


### Sites que realizaremos as buscas

In [ ]:
URL_LINKANA = "https://cnpj.linkana.com/"
URL_CNPJBIZ = "https://cnpj.biz/empresas"
URL_ECONODATA = "https://www.econodata.com.br/consulta-empresa"

### Recuperando String de Print da Tela

In [ ]:
import easyocr
from PIL import Image

def recuperando_string_print(x1, y1, width, height):
    '''
    Captura um print da tela e transforma em string
    '''
    # Inicializa o EasyOCR para o idioma português
    reader = easyocr.Reader(['pt'])

    # Captura uma região da tela onde o texto está localizado (ajuste as coordenadas conforme necessário)
    screenshot = pyautogui.screenshot(region=(x1, y1, width, height))

    # Salva a captura de tela em um arquivo temporário
    screenshot_path = "screenshot_temp.png"
    screenshot.save(screenshot_path)

    # Usa EasyOCR para realizar OCR na imagem
    resultados = reader.readtext(screenshot_path)
    
    # Extrai o texto dos resultados
    texto_copiado = ""
    for resultado in resultados:
        texto_copiado += resultado[1] + " "
    return texto_copiado.strip()


### Verificar CNPJ no CRESCI - RJ

In [ ]:
def verificar_cnpj_cresci(cnpj, inscricao_empresa):
    # empresa = {informações da linha do dataframe}
    # Entrar no Cresci
    driver = iniciando_edge_selenium('https://www.crecirj.conselho.net.br/form_pesquisa_cadastro_geral_site.php')
    # Digitar o CNPJ
    campo_cnpj = driver.find_element("id", "input-21")
    campo_cnpj.send_keys(cnpj)

    # Envie o formulário
    campo_cnpj.send_keys(Keys.RETURN)
    time.sleep(2)
    # Verificar se é compativel
    try:
        # Encontra a tabela na página
        table = driver.find_elements('css selector', 'table')
        table_html = table[0].get_attribute('outerHTML')
    except IndexError:
        print(f"Não existe nenhuma corretora/imobiliaria com o cnpj: {cnpj}!")
        driver.quit()
        return False
    else:
        df = pd.read_html(StringIO(table_html))[0]  # Este método retorna uma lista de DataFrames, portanto, pegamos o primeiro
        driver.quit()
        # Recuperando dados da empresa
        empreasa_info = df.iloc[0]
        empresa_inscricao = empreasa_info["Inscrição"]

        if empresa_inscricao == inscricao_empresa:
            return True
        else:
            print(f"A inscriçãoda da empresa {inscricao_empresa} é diferente de {empresa_inscricao}")
            return False


### Buscar no CNPJ.BIZ

#### Pesquisar empresa pelo nome

In [ ]:
def pesquisar_empresa_nome_cnpjbiz(nome_empresa):
    '''
    Pesquisa uma empresa pelo nome
    '''
    pyperclip.copy(nome_empresa)
    time.sleep(1)
    pyautogui.click(1062, 409)
    pyautogui.hotkey("ctrl", "v")
    time.sleep(1)
    pyautogui.press("enter")
    time.sleep(1)
    

#### Verificar se existe empresa

In [ ]:
def verificar_existe_empresa_cnpjbiz():
    '''
    Verifica se ao pesquisar a empresa, sera exibido uma mensagem de erro
    Caso não seja encontrado nada sera retornado verdadeiro, caso o contrario falso
    '''
    deu_ruim = recuperando_string_print(639, 235, 500, 300)
    time.sleep(2)
    if "Opa! Alguma coisa saiu errado" in deu_ruim:
        print("Deu ruim")
        print(deu_ruim)
        return False
    else:
        return True

#### Capturar CNPJ da empresa

In [ ]:
def capturar_cnpj_empresa_cnpjbiz():
    '''
    Captura um print da tela onde o cnpj deve estar. 
    E transforma essa imagem em texto
    '''
    pyautogui.PAUSE = .5
    for _ in range(5):
        pyautogui.press("down")
    pyautogui.PAUSE = 1

    pyautogui.moveTo(445,430)
    texto = recuperando_string_print(445, 430, 300, 200).split(" ")[0]
    return texto

#### Acessa a Pagina da empresa e inicia um driver no selenium com essa url

In [ ]:
def acessando_empresas_info_to_selenium_cnpjbiz():
    '''
    Acessar o site com as informações da empresa e criar um driver no selenium com essa mesma url
    '''
    pyautogui.click(445,430)
    time.sleep(1)
    pyautogui.click(581,62)
    time.sleep(1)
    pyautogui.hotkey("ctrl", "c")

    time.sleep(1)
    link = pyperclip.paste()
   
    pyautogui.click(1888, 18)
    return iniciando_edge_selenium(link)
    

#### Recuperando Contatos

In [ ]:
def recuperando_contatos_cnpj_biz(driver:webdriver.Edge):
    # Digitar o CNPJ
    h2_contatos = driver.find_element(By.ID, "contato")
    p_contatos = h2_contatos.find_element(By.XPATH, "following-sibling::p")
    contatos = p_contatos.find_elements(By.TAG_NAME, "b")
    return ", ".join([contato.text for contato in contatos])

#### Recuperando Socios

In [ ]:
def recuperando_socios_cnpj_biz(driver:webdriver.Edge):
    elementos_h2 = driver.find_elements(By.TAG_NAME, "h2")
    if len(elementos_h2) >= 5:
        h2 = elementos_h2[4]
        p = h2.find_element(By.XPATH, "following-sibling::p")
        socios = p.text.split("\n")
    return ", ".join(socios)

#### Recuperando Endereco

In [ ]:
def recuperando_endereco_cnpj_biz(driver:webdriver.Edge):
    elementos_h2 = driver.find_elements(By.TAG_NAME, "h2")
    if len(elementos_h2) >= 3:
        h2 = elementos_h2[2]
        p = h2.find_elements(By.XPATH, "following-sibling::p[7]")[0]
        #b = p.find_element(By.TAG_NAME, "b")
        endereco = p.text.split("\n")
        endereco.pop(0)
    return ", ".join(endereco)

#### Listar elementos html !!!!!!!!!!!!!!!!!!

In [ ]:
def listar_elementos_pagina(driver:webdriver.Edge, elemento):
    elementos = driver.find_elements(By.XPATH, f"following-sibling::{elemento}")
    for idx, element in enumerate(elementos):
        print(idx, element.text)
    else:
        print(f"Não possui nenhum elemento <{elemento}/>")
    return elementos

#### Buscar dados no CNPJ.Biz

In [ ]:
def buscar_empresa_cnpjbiz(nome_empresa, inscricao_empresa):
    pyautogui.PAUSE = 1
    iniciando_edge_pyautogui(URL_CNPJBIZ)
    pesquisar_empresa_nome_cnpjbiz(nome_empresa)
    
    existe_empresa = verificar_existe_empresa_cnpjbiz()
    
    if existe_empresa:
    
        cnpj = capturar_cnpj_empresa_cnpjbiz()
        cnpj_valido = verificar_cnpj_cresci(cnpj, inscricao_empresa)
    
        if cnpj_valido:
            print("CNPJ Valido Porra")
            driver = acessando_empresas_info_to_selenium_cnpjbiz()
            contatos = recuperando_contatos_cnpj_biz(driver)
            socios = recuperando_socios_cnpj_biz(driver)
            endereco = recuperando_endereco_cnpj_biz(driver)
            print("Contatos: ", contatos)
            print("Socios: ", socios)
            print("Endereco: ", endereco)
            return {"Contatos":contatos, "Socios": socios, "Endereço": endereco}    
        else:
            print("FUdeu deu ruim")
        
    return False
#cnpj_empresa = buscar_empresa_cnpjbiz("2M INTERMEDIAÇÕES LTDA", "10663")

In [ ]:
def teste():
    #driver = iniciando_edge_selenium("https://cnpj.biz/24701445000167")
    #h2 = listar_elementos_pagina(driver, "h2")[2]
    #p = listar_elementos_pagina(h2, "p")

    #'''
    driver = iniciando_edge_selenium("https://cnpj.biz/24701445000167")
    contatos = recuperando_contatos_cnpj_biz(driver)
    socios = recuperando_socios_cnpj_biz(driver)
    endereco = recuperando_endereco_cnpj_biz(driver)
    print("Contatos: ", contatos)
    print("Socios: ", socios)
    print("Endereco: ", endereco)
    #'''

#teste()

### Buscar no Linkana

#### Buscar no Linkana pelo nome da Empresa

In [ ]:
def pesquisar_empresa_nome_linkana(nome_empresa, driver):
    campo_nome = driver.find_element(By.ID, "q")
    campo_nome.send_keys(nome_empresa)
    campo_nome.send_keys(Keys.RETURN)

#### Verificando se existe empresa

In [ ]:
def verificar_existe_empresa_linkana():
    texto = recuperando_string_print(313, 529, 500, 70)
    print(texto)
    return not "Nenhum resultado encontrado" in texto
    

#### Recuperando Contato

In [ ]:
def recuperando_contatos_linkana(driver:webdriver.Edge):
    lista = driver.find_elements(By.TAG_NAME, "ul")[0].text.split("\n")
    email = lista[21]
    telefone = lista[23]
    contato = [email,telefone]
    print(f"Contato: {contato}")
    return contato



#### Recuperando Socios

In [ ]:
def recuperando_socios_linkana(driver:webdriver.Edge):
    lista = driver.find_elements(By.TAG_NAME, "ul")[1].text.split("\n")
    socios = []
    for idx, item in enumerate(lista):
        if idx % 2 == 0:
            print(f"{idx/2 + 1} - {item}")
            socios.append(item)
    return socios

#### Recuperando localização

In [ ]:
def recuperando_endereco_linkana(driver:webdriver.Edge):
    lista = driver.find_elements(By.TAG_NAME, "ul")[3].text.split("\n")
    step = ""
    endereco = []
    for idx, item in enumerate(lista):
        if idx % 2 == 0:
            step = item
        else:
            endereco.append([step, item])
    print(endereco)

#### Recuperando Endereco

In [ ]:
def buscar_empresa_linkana(nome_empresa, inscricao_empresa):
    driver = iniciando_edge_selenium(URL_LINKANA)
    
    pesquisar_empresa_nome_linkana(nome_empresa, driver)

    existe_empresa = verificar_existe_empresa_linkana()
    if existe_empresa:
        cnpj = recuperando_string_print(337, 584, 250, 60).split(" ")[1]
        cnpj_valido = verificar_cnpj_cresci(cnpj, inscricao_empresa)
        if cnpj_valido:
            pyautogui.click(337,584)
            time.sleep(1)
            contatos = recuperando_contatos_linkana(driver)
            socios = recuperando_socios_linkana(driver)
            enderecos = recuperando_endereco_linkana(driver)
            return {"Contatos":contatos, "Socios": socios, "Endereço": enderecos}    
    else:
        print("Nenhuma empresa encontrada")
        
    return False

#buscar_empresa_linkana("2F IMOBILIARIA LTDAR", "10591")

In [ ]:
'''
def teste_2():
    driver = iniciando_edge_selenium("https://cnpj.linkana.com/cnpj/2f-imobiliaria-ltda/51884181000105")
    recuperando_contatos_linkana(driver)

teste_2()'''
    

### Buscar no CNPJ INFO

#### Pesquisar empresa no cnpj info

In [ ]:
def pesquisar_empresa_nome_cnpjinfo(nome_empresa, driver):
    campo_empresa = driver.find_elements(By.CSS_SELECTOR, "h3 input")[0]
    campo_empresa.send_keys(nome_empresa)
    campo_empresa.send_keys(Keys.RETURN)

#### Recuperar CNPJ no cnpj info

In [ ]:
def recuperar_cnpj_cnpjinfo(driver):
    content = driver.find_element(By.ID, "content")
    lista_empresas = content.find_elements(By.TAG_NAME, "ul")[0]
    itens_lista_empresas = lista_empresas.find_elements(By.TAG_NAME, "li")
    primeiro_item = itens_lista_empresas[0]
    cnpj = primeiro_item.find_elements(By.TAG_NAME, "a")[0]
    return cnpj


#### Recuperar endereço

In [ ]:
def recuperando_endereco_cnpjinfo(driver):
    content = driver.find_element(By.ID, "content")
    content_text = content.text
    endereco = content_text.split("\nEndereço\n")[1].split("\nContatos\n")[0]
    print(endereco)
    return endereco

#### Recuperar contatos

In [ ]:
def recuperando_contatos_cnpjinfo(driver:webdriver.Edge):
    content = driver.find_element(By.ID, "content").text
    contatos = content.split("\nTelefone(s): ")[1].split("\nSócios\n")[0].strip().split(" e ")
    print(contatos)
    return contatos
    

#### Recuperando Socios

In [ ]:
def recuperando_socios_cnpjinfo(driver:webdriver.Edge):
    content = driver.find_element(By.ID, "content").text
    socios = content.split("Sócios\nCódigo Nome Data de entrada Qualificação\n")[1].split("Atividades de negócios da empresa")[0].split("\n")
    print(socios)

In [ ]:
def buscar_empresa_cnpjinfo(nome_empresa, inscricao_empresa):
    driver = iniciando_edge_selenium("http://cnpj.info/")
    pesquisar_empresa_nome_cnpjinfo(nome_empresa, driver)
    cnpj = recuperar_cnpj_cnpjinfo(driver)
    cnpj_valido = verificar_cnpj_cresci(cnpj.text, inscricao_empresa)
    if cnpj_valido:
        cnpj.click()
        endereco = recuperando_endereco_cnpjinfo(driver)
        contatos = recuperando_contatos_cnpjinfo(driver)
        socios = recuperando_socios_cnpjinfo(driver)
        return {"Contatos":contatos, "Socios": socios, "Endereço": endereco}       
    else:
        print("CNPJ INVALIDO")

    return False

#buscar_empresa_cnpjinfo("2M INTERMEDIAÇÕES LTDA", "10663")

In [ ]:
def teste():
    driver = iniciando_edge_selenium("http://cnpj.info/24701445000167")
    time.sleep(5)
    #content = driver.find_element(By.ID, "content")
    recuperando_socios_cnpjinfo(driver)
    #print(content.text)

#teste()

### Realizar Busca Completa

#### Listas Empresas

In [ ]:
lista_empresas_informacoes_validas = []
lista_empresas_sem_informacoes = []

#### Buscar empresa

In [ ]:
def buscar_dados_empresa(empresa, indice, tb:pd.DataFrame):

    nome_empresa = empresa["Identificação"]
    inscricao_empresa = empresa["Inscrição"]
    print("Nome Empresa:", nome_empresa)
    print("Inscricao Empresa", inscricao_empresa)

    dados_novos = {}

    recuperou = buscar_empresa_cnpjbiz(nome_empresa, inscricao_empresa)
    if not recuperou:
        recuperou = buscar_empresa_linkana(nome_empresa, inscricao_empresa)
        if not recuperou:
            recuperou = buscar_empresa_cnpjinfo(nome_empresa, inscricao_empresa)
            if not recuperou:
                lista_empresas_sem_informacoes.append(empresa)
            else:
                dados_novos.update(recuperou)
        else:
            dados_novos.update(recuperou)
    else:
        dados_novos.update(recuperou)

    if recuperou:
        for idx, valor in dados_novos.items():
            tb.at[indice, idx] = valor

In [ ]:
#tijuca = dict_bairros_edit["Tijuca"]
#print(tijuca.iloc[0])
#tijuca.at[0, "Identificação"] = "2M INTERMEDIAÇÕES LTDA"
#buscar_dados_empresa(tijuca.iloc[0], 0, tijuca)

#print(tijuca.iloc[0])
def buscar_todas_empresas_bairro(df:pd.DataFrame):
    novo_df = pd.DataFrame(columns=list(df.columns))
    for idx, empresa in df.iterrows():
        buscar_dados_empresa(empresa, idx, novo_df)
    return novo_df

In [ ]:
#buscar_todas_empresas_bairro()

#print(dict_bairros_edit["Tijuca"].iloc[0])

## Conectar Google Sheets 

### Importando dependências

In [46]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from gspread_dataframe import set_with_dataframe
import gspread

### Criar Planilhas no GOOGLE SHEETS

In [ ]:
def criar_planilha_gs(nome, service):
    # Create the spreadsheet
    planilha = {"properties": {"title": nome}}
    planilha = (
        service.spreadsheets()
        .create(body=planilha, fields="spreadsheetId")
        .execute()
    )
    spreadsheet_id = planilha.get("spreadsheetId")
    print(f"Spreadsheet ID: {spreadsheet_id}")
    return spreadsheet_id

### Ler Planilhas no GOOGLE SHEETS

In [ ]:
def ler_planilha(id_planilha, service, range="Página1!A:Z"):
    planilha = service.spreadsheets().values().get(spreadsheetId=id_planilha, range=range).execute()
    print(planilha['values'])
    return planilha

### Editar planilhas no GOOGLE SHEETS

In [45]:
def editar_planilha(id_planilha, service, novo_conteudo, range="Pagina1!A"):
    planilha = service.spreadsheets().values().update(
        spreadsheetId=id_planilha,
        range=range, valueInputOption="USER_ENTERED",
        body={'values':novo_conteudo}
    ).execute()
    return planilha

### Transforma Dataframe em Lista

In [44]:
def transforma_dataframe_em_lista(dataframe:pd.DataFrame, add_columns=False):
    nova_lista = []
    columns = list(dataframe.columns)
    if add_columns:
        nova_lista.append(columns)
    for index, linha in dataframe.iterrows():
        itens_lista = []
        for item in linha:
            itens_lista.append(item)
        nova_lista.append(itens_lista)
    return nova_lista

### Validar Token GOOGLE SHEETS

In [43]:
SCOPES = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets"]

def validar_token_google_sheets():
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "client_secret.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())
  return creds

### Transformar Dataframe em planilha

In [52]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
id_planilha = "1AeDuHW7f4-7vBVCgQvMBYqx2ZCTxcO7D0qbzMZ2l8Ps"

def transformar_dataframe_planilha(title, dataframe):
    creds = validar_token_google_sheets()
    ordem_df = ["Inscrição", "Identificação", "Situação", "Certidão de Regularidade", "Contato", "Natureza", "Bairro"]	
    if all(coluna in dataframe.columns for coluna in ordem_df):
        dataframe = dataframe[ordem_df]																	
        service = build("sheets", "v4", credentials=creds)
        conteudo_para_adicionar = transforma_dataframe_em_lista(dataframe, add_columns=True)
        editar_planilha(id_planilha, service, conteudo_para_adicionar, range=f"{title}!A1:Z")

    print(f"Planilha Editada - {title}")

In [61]:
for bairro, tb in dict_bairros_edit.items():
    transformar_dataframe_planilha(bairro, tb)

Planilha Editada - Jardim Guanabara
Planilha Editada - Maré
Planilha Editada - Moneró
Planilha Editada - Olaria
Planilha Editada - Pitangueiras
Planilha Editada - Portuguesa
Planilha Editada - Praia da Bandeira
Planilha Editada - Ramos
Planilha Editada - Ribeira
Planilha Editada - Tauá
Planilha Editada - Zumbi
Planilha Editada - Alto da Boa Vista
Planilha Editada - Andaraí
Planilha Editada - Grajaú
Planilha Editada - Maracanã
Planilha Editada - Praça da Bandeira
Planilha Editada - Tijuca
Planilha Editada - Vila Isabel
Planilha Editada - Barra da Tijuca
Planilha Editada - Abolição
Planilha Editada - Agua Santa
Planilha Editada - Cachambi
Planilha Editada - Del Castilho
Planilha Editada - Encantado
Planilha Editada - Engenho de Dentro
Planilha Editada - Engenho Novo
Planilha Editada - Higienópolis
Planilha Editada - Jacaré
Planilha Editada - Jacarezinho
Planilha Editada - Lins de Vasconcelos
Planilha Editada - Manguinhos
Planilha Editada - Maria da Graça
Planilha Editada - Méier
Planilha

### Recuperar planilha bairros

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

def baixar_pagina_planilha(nome_planilha, nome_pagina):
  creds = validar_token_google_sheets()
    # Substitua pelo nome do seu arquivo de credenciais

  # Autorize e abra a planilha
  client = gspread.authorize(creds)
  spreadsheet = client.open(nome_planilha)  # Substitua pelo nome da sua planilha
  worksheet = spreadsheet.worksheet(nome_pagina)  # Substitua "Tijuca" pelo nome da sua página

  # Obtenha os dados da planilha como uma lista de dicionários
  data = worksheet.get_all_records()

  # Converta os dados em um DataFrame do Pandas
  df = pd.DataFrame(data)

  # Agora você pode usar df para trabalhar com seus dados
  return df

### Remover PF

In [62]:
def remover_empresas_pf(df):
    novo_df = pd.DataFrame(columns=list(df.columns))
    for idx, empresa in df.iterrows():
        print(empresa["Natureza"])
        if "PJ" in empresa["Natureza"]:
            novo_df.append(empresa)
        
    return novo_df


### Atualizar Planilha Google Sheets

In [63]:
def remover_planilha_adicionar_df(nome_planilha, nome_pagina, novo_df):
    creds = validar_token_google_sheets()
    client = gspread.authorize(creds)
    spreadsheet = client.open(nome_planilha)
    worksheet = spreadsheet.worksheet(nome_pagina)

    # Limpar os dados na planilha
    worksheet.clear()
    # Atualizar a planilha com os dados atualizados
    novo_df_list = transforma_dataframe_em_lista(novo_df, add_columns=True)
    worksheet.append_rows(novo_df_list)
    print("Planilha atualizada")